# Verificando o conteúdo do dataset

> ação:
> 1. Verificar necessidade de limpeza dos dados (texo).  a data precisaremos separar o horario
> 2. Verificar qual coluna do texto vamos utilizar
> 3. Estudar a polaridade das palavras
> 4. Verificar palavras mais utilizadas
> 5. Buscar dados econômicos e o price action do EURUSD
> 6. Cenas dos próximos insights

## Começou!

In [1]:
#import das bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('C:\Python\Mentoria\Forex\Forex\Dataset\compilado\cnbc_dataset.csv')
df.sample(3)

,title,published_at,publisher,short_description,description
364,'Rolling Stone' Boston bomber photo stirs deba...,2013-07-17 20:28:49+00:00,CNBC,Rolling Stone magazine sparked a heated social...,Rolling Stone magazine sparked a heated social...
217,Crypto investor who bought Beeple's NFT for $6...,2021-03-30 12:27:27+00:00,CNBC,The buyer of the $69 million NFT by the artist...,The buyer of the $69 million NFT by the artist...
472,"Wells Fargo rates Ford, GM as overweight, prai...",2021-04-05 14:11:33+00:00,CNBC,Legacy automakers Ford and General Motors are ...,Legacy automakers Ford and General Motors are ...


## Verificando a polaridade da coluna 'title' utilizando o textblob

TextBlob aims to provide access to common text-processing operations through a familiar interface. You can treat TextBlob objects as if they were Python strings that learned how to do Natural Language Processing.

https://textblob.readthedocs.io/en/dev/quickstart.html

### Sentiment Analysis
The sentiment property returns a namedtuple of the form Sentiment(polarity, subjectivity). 

The polarity score is a float within the range [-1.0, 1.0]. 

The subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.

>>> testimonial = TextBlob("Textblob is amazingly simple to use. What great fun!")

>>> testimonial.sentiment

Sentiment(polarity=0.39166666666666666, subjectivity=0.4357142857142857)

>>> testimonial.sentiment.polarity

0.39166666666666666

In [ ]:
pip install textblob
pip install --upgrade pip

In [11]:
from textblob import TextBlob

In [45]:
df['sentiment'] = df['title'].apply(lambda x: TextBlob(x).sentiment_assessments)

In [47]:
df.head(10)

,title,published_at,publisher,short_description,description,sentiment
0,Santoli’s Wednesday market notes: Could Septem...,2021-09-29 17:09:39+00:00,CNBC,"This is the daily notebook of Mike Santoli, CN...","This is the daily notebook of Mike Santoli, CN...","(0.0, 0.0, [(['fourth'], 0.0, 0.0, None)])"
1,My take on the early Brexit winners and losers,2016-06-24 13:50:48-04:00,CNBC,This commentary originally ran on Facebook. Bo...,NaN,"(-0.05, 0.25, [(['early'], 0.1, 0.3, None), ([..."
2,Europe&#039;s recovery depends on Renzi&#039;s...,2014-03-25 13:29:45-04:00,CNBC,"In spring, ambitious reforms began in Italy. U...",NaN,"(0.0, 0.0, [])"
3,US Moves Closer to Becoming A Major Shareholde...,2009-04-22 19:49:03+00:00,CNBC,The US government is increasingly likely to co...,The US government is increasingly likely to co...,"(0.25625, 0.675, [(['becoming'], 0.45, 0.85000..."
4,Trump: 'Mission accomplished' on 'perfectly ex...,2018-04-14 14:59:04+00:00,CNBC,NaN,President Donald Trump hailed the U.S.-led int...,"(0.6, 0.75, [(['accomplished'], 0.2, 0.5, None..."
5,Chevron CEO Watson says he supports Trump on t...,2017-03-07 23:07:14+00:00,CNBC,Chevron Chief Executive John Watson told CNBC ...,Chevron Chief Executive John Watson told CNBC ...,"(0.0, 0.0, [])"
6,22. Hexadite,2017-02-28 11:00:37-05:00,CNBC,"Founders: Eran Barak, Barak Klinghofer (chief ...",NaN,"(0.0, 0.0, [])"
7,European stocks close higher on supportive Fed...,2020-12-17 06:08:38+00:00,CNBC,LONDON — European stocks closed higher on Thur...,LONDON — European stocks closed higher on Thur...,"(0.25, 0.5, [(['european'], 0.0, 0.0, None), (..."
8,"Fewer investors have a 'fear of missing out,' ...",2019-07-28 21:00:47+00:00,CNBC,Oppenheimer Asset Management's John Stoltzfus ...,Oppenheimer Asset Management's John Stoltzfus ...,"(-0.2, 0.05, [(['missing'], -0.2, 0.05, None)])"
9,Morgan Stanley Tries to Stave Off Ratings Cut,2012-04-05 06:53:57+00:00,CNBC,"James Gorman, Morgan Stanley’s chief executive...","James Gorman, Morgan Stanley’s chief executive...","(-0.1, 0.4, [(['tries'], -0.1, 0.4, None)])"


In [52]:
#podemos fazer assim tb: (obrigada, StackOverFlow!)
def sentiment_calc(text):
    try:
        return TextBlob(text).sentiment_assessments
    except:
        return None

df['sentiment'] = df['title'].apply(sentiment_calc)

In [53]:
df

,title,published_at,publisher,short_description,description,sentiment
0,Santoli’s Wednesday market notes: Could Septem...,2021-09-29 17:09:39+00:00,CNBC,"This is the daily notebook of Mike Santoli, CN...","This is the daily notebook of Mike Santoli, CN...","(0.0, 0.0, [(['fourth'], 0.0, 0.0, None)])"
1,My take on the early Brexit winners and losers,2016-06-24 13:50:48-04:00,CNBC,This commentary originally ran on Facebook. Bo...,NaN,"(-0.05, 0.25, [(['early'], 0.1, 0.3, None), ([..."
2,Europe&#039;s recovery depends on Renzi&#039;s...,2014-03-25 13:29:45-04:00,CNBC,"In spring, ambitious reforms began in Italy. U...",NaN,"(0.0, 0.0, [])"
3,US Moves Closer to Becoming A Major Shareholde...,2009-04-22 19:49:03+00:00,CNBC,The US government is increasingly likely to co...,The US government is increasingly likely to co...,"(0.25625, 0.675, [(['becoming'], 0.45, 0.85000..."
4,Trump: 'Mission accomplished' on 'perfectly ex...,2018-04-14 14:59:04+00:00,CNBC,NaN,President Donald Trump hailed the U.S.-led int...,"(0.6, 0.75, [(['accomplished'], 0.2, 0.5, None..."
...,...,...,...,...,...,...
620,Cash-Hungry States Are Putting Buildings on th...,2010-05-05 17:16:33+00:00,CNBC,Is it better to rent or to own?The default ans...,Is it better to rent or to own?The default ans...,"(0.0, 0.0, [])"
621,Since 2009 every time stocks have 3-week losin...,2019-05-29 15:11:51+00:00,CNBC,Stocks began the truncated trading week after ...,Stocks began the truncated trading week after ...,"(0.0, 0.0, [(['next'], 0.0, 0.0, None)])"
622,Noble Energy Announces Conference Call To Disc...,2012-10-05 14:59:00+00:00,CNBC,NaN,"HOUSTON, Oct. 5, 2012 /PRNewswire/ -- Noble En...","(0.36818181818181817, 0.6772727272727272, [(['..."
623,Here are some ways to play the 'stay-at-home' ...,2020-05-02 16:57:58+00:00,CNBC,"With millions working and learning from home, ...","With millions working and learning from home, ...","(0.0, 0.0, [])"
